# **1. Importación de *modules***

In [3]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config Completer.use_jedi = False

import warnings
warnings.filterwarnings("ignore")

import funciones as f

# **2. Importación del dataset patient_covid.csv**

In [5]:
data = pd.read_csv("../data/patient_covid.csv", sep = ",", low_memory=False) 
f._get_info(data)

   sex  patient_type  intubated  pneumonia  age  pregnant  diabetes  copd  \
0    2             1         97          2   42        97         2     2   
1    1             1         97          2   51         2         2     2   
2    2             2          2          2   51        97         1     2   

   asthma  immunosuppression  hypertension  other_diseases  cardiovascular  \
0       1                  2             2               2               2   
1       2                  2             2               2               2   
2       2                  2             1               2               2   

   obesity  chronic_kidney_failure  smoker  another_case  outcome  icu  \
0        2                       2       2            99        1   97   
1        2                       2       2            99        1   97   
2        1                       2       2            99        1    2   

   death_date  
0  9999-99-99  
1  9999-99-99  
2  9999-99-99  
(95839, 20)


# **3. Tratamiento de los datos**

# **4.Modelos**

## 4.1. Regresión logística (Marce y Dani)

## 4.2. KNN (Mae)

### 4.2.1. Búsqueda y selección del hiperparámetro

### 4.1. Naive Bayes multinomial (Enzo)


In [1]:
## 4.1.1. Categorización de edad (o no :/)